#### Name: Anush Ravishankar 
#### Registration No: 200968250
#### AI - Week 9

In [ ]:
import gym
import numpy as np


In [ ]:
# Define the Cliff Walking environment
env = gym.make('CliffWalking-v0')

/usr/local/lib/python3.9/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.9/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [ ]:
# Set the number of episodes
num_episodes = 100

# Set the discount factor
gamma = 1.0

# Define the epsilon value for epsilon-soft policies
epsilon = 0.1

# Define the number of actions and states
num_actions = env.action_space.n
num_states = env.observation_space.n


In [ ]:
# Define the Q-value table for Monte Carlo ES
Q_mc_es = np.zeros((num_states, num_actions))
returns_sum = np.zeros((num_states, num_actions))
returns_count = np.zeros((num_states, num_actions))

# Define the Q-value table for on-policy first-visit MC control
Q_on_policy = np.zeros((num_states, num_actions))
returns_sum_on_policy = np.zeros((num_states, num_actions))
returns_count_on_policy = np.zeros((num_states, num_actions))


In [ ]:
# Implement Monte Carlo ES algorithm
for i_episode in range(num_episodes):
    episode = []
    state = env.reset()
    done = False
    
    # Generate an episode using exploring starts
    while not done:
        action = np.random.choice(num_actions)
        next_state, reward, done, info = env.step(action)
        episode.append((state, action, reward))
        state = next_state
    
    # Update the Q-value table
    states, actions, rewards = zip(*episode)
    discounts = np.array([gamma**i for i in range(len(rewards)+1)])
    
    # Calculate the returns
    for i, state in enumerate(states):
        action = actions[i]
        returns_sum[state][action] += sum(rewards[i:] * discounts[:-(1+i)])
        returns_count[state][action] += 1.0
        Q_mc_es[state][action] = returns_sum[state][action] / returns_count[state][action]

In [ ]:
# Define the epsilon-soft policy
def epsilon_soft_policy(state, Q, epsilon):
    policy = np.ones(num_actions, dtype=float) * epsilon / num_actions
    best_action = np.argmax(Q[state])
    policy[best_action] += (1.0 - epsilon)
    return policy

In [ ]:
# Implement on-policy first-visit MC control algorithm
for i_episode in range(num_episodes):
    episode_states = []
    episode_actions = []
    episode_rewards = []
    state = env.reset()
    done = False
    
    # Generate an episode using epsilon-soft policy
    while not done:
        policy = epsilon_soft_policy(state, Q_on_policy, epsilon)
        action = np.random.choice(num_actions, p=policy)
        next_state, reward, done, info = env.step(action)
        episode_states.append(state)
        episode_actions.append(action)
        episode_rewards.append(reward)
        state = next_state
    
    # Update the Q-value table
    discounts = gamma ** np.arange(len(episode_rewards) + 1)
    returns = np.cumsum(episode_rewards[::-1])[::-1] * discounts[:-1]
    for t in range(len(episode_states)):
        state = episode_states[t]
        action = episode_actions[t]
        returns_sum_on_policy[state][action] += returns[t]
        returns_count_on_policy[state][action] += 1
        Q_on_policy[state][action] = returns_sum_on_policy[state][action] / returns_count_on_policy[state][action]

In [ ]:
# Print the optimal policy for each algorithm
print("Optimal Policy using Monte Carlo ES:\n", np.argmax(Q_mc_es, axis=1))
print("\nOptimal Policy using on-policy first-visit MC control:\n", np.argmax(Q_on_policy, axis=1))

Optimal Policy using Monte Carlo ES:
 [1 3 1 1 1 1 0 2 1 0 1 2 2 1 1 1 1 1 1 1 1 1 1 2 3 3 3 0 1 1 1 0 3 1 1 2 0
 0 0 0 0 0 0 0 0 0 0 0]

Optimal Policy using on-policy first-visit MC control:
 [3 3 0 2 3 0 2 2 2 2 1 1 2 3 1 1 1 1 1 1 1 1 1 2 1 3 2 1 1 0 1 3 1 2 1 2 2
 0 0 0 0 0 0 0 0 0 0 0]


#### Compare  and comment  on  the  performance  of  both  methods  in  terms  of  number  of  steps needed to learn optimal policy and the number of episodes

Monte Carlo ES method generally requires a large number of episodes to converge to the optimal policy. This is because it needs to visit each state-action pair sufficiently often to estimate the Q-values accurately. 

Therefore, it may require a large number of episodes to learn the optimal policy.

___________ 

On-policy first-visit MC control for epsilon-soft policies generates episodes using an epsilon-soft policy, which means that it selects the best action with probability = (1-epsilon) and a random action with probability epsilon. This helps the algorithm to explore the state-action space while also exploiting the current best policy. 

The on-policy first-visit MC control method generally requires fewer episodes to converge compared to Monte Carlo ES method. This is because it gradually improves the policy while generating episodes and updates the Q-values after each episode.
____
In terms of the number of steps needed to learn the optimal policy, both methods should learn the optimal policy in a similar number of steps, assuming that they have converged to the optimal policy. 

However, the number of steps required for convergence may vary depending on the specific problem and the hyperparameters used.